In [2]:
import os
import json
import requests
import time
import random
import csv
from openai import OpenAI
from dotenv import load_dotenv

In [4]:
# Accessing OpenAI API
load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv("key")
client = OpenAI()

In [46]:
# Function to download generated images from the provided url
def download_image(url, filename):
    response = requests.get(url)
    if response.status_code == 200:
        with open(filename, 'wb') as f:
            f.write(response.content)
        print(f"Image downloaded successfully as {filename}")
    else:
        print(f"Failed to download image from {url}")

# Creating list
def create_list(list_prompt, temp=0.7):
  chat_response = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    response_format={ "type": "json_object" },
    temperature=temp,
    messages=[
      {"role": "system", "content": "You are a helpful assistant designed to output JSON."},
      {"role": "user", "content": list_prompt}
    ]
  )
  data = json.loads(chat_response.choices[0].message.content)
  data = list(data.values())[0]
  data = list(set(data))
  return data

# Generate pics with Dall-e-2
def create_pics(profile_prompt, filename=None):
  img_response = client.images.generate(
    model="dall-e-2",
    prompt=profile_prompt,
    size="256x256",
    quality="standard",
    n=1,
  )
  image_url = img_response.data[0].url

  if filename is None:
    filename = image_url[-15:]

  filepath ="images/"+  filename+ ".jpg"
  download_image(image_url, filepath)
  time.sleep(8)
  return filepath



In [42]:
# Create accounts
def create_accounts(users):
    for user in users:
        new_user =  {"username": user, "password": user}
        newuser_response = requests.post("http://localhost:8000/server/newuser", json=new_user)

# Uploading posts
def upload_posts(csvfile):
    user_posts = dict()
    with open(csvfile, "r") as file:
        csv_reader = csv.reader(file)

        for row in csv_reader:
            user_posts[row[0]] = user_posts.get(row[0], []) + [row[1]]

    for user in user_posts.keys():
        login =  {"username": user, "password": user}
        res = requests.post("http://localhost:8000/server/login", json=login)

        for post_text in user_posts[user]:
            current_time = time.time()*1000 - random.randint(0,999999999)
            post = {"text": post_text, "image":"", "time": current_time}
            res2 = requests.post("http://localhost:8000/server/newpost", json=post, cookies=res.cookies)

# Uploading profile pics
def upload_profile_pics(user_pics):
    for user in user_pics.keys():
        login =  {"username": user, "password": user}
        res = requests.post("http://localhost:8000/server/login", json=login)
        
        with open(user_pics[user], "rb") as file:
            files = {"image": file}
            res2 = requests.post("http://localhost:8000/server/images", files=files, cookies=res.cookies)


In [51]:
# Create usernames, profile pic, and posts
usernames = create_list("Generate a list of 40 different first names.")
animals = create_list("Generate a list of 40 different animals that are found in North America or the arctic.")
assert len(usernames)==len(animals), "Length of lists do not match"

user_img_paths = {}
user_posts = {}
for i, user in enumerate(usernames):
    try:
        path = create_pics(f"A cartoon image of a {animals[i]} with its face in the center of the image")
        user_img_paths[user] = path
        count = random.randint(7,15)
        post_prompt = f"""Pretend you are a {animals[i]}. Generate a list of {count} social media posts. 
        Each item in the list should only consist of the text of the post."""
        user_posts[user] = create_list(post_prompt, temp=1)
    except:
         print("Error with:", user, animals[i])

with open ("posts.csv", "w", newline="") as file:
        csvwriter = csv.writer(file)
        for user in usernames:
            if user in user_posts:
                for post in user_posts[user]:
                    csvwriter.writerow([user, post])

Image downloaded successfully as images/6PdlPpQ%3D.jpg


In [53]:
# Create accounts and upload posts
create_accounts(usernames)
upload_profile_pics(user_img_paths)
upload_posts("posts.csv")

<Response [200]>
<Response [200]>
